In [1]:
try:
    with open('i.html') as fp:
        text = fp.read()
except UnicodeDecodeError as exc:
    print(exc)

'utf-8' codec can't decode byte 0x80 in position 1721: invalid start byte


In [2]:
with open('i.html', 'rb') as fp:
    octets = fp.read()

for i in range(1715, 1725):
    code = octets[i]
    octet = octets[i:i+1]
    char = octet.decode('cp1252')
    print(f'{code:02x} {char!r}')

0a '\n'
0a '\n'
3c '<'
21 '!'
2d '-'
2d '-'
80 '€'
80 '€'
80 '€'
80 '€'


In [3]:
with open('i.html', encoding='cp1252') as fp:
     text = fp.read()

In [ ]:
import re
import html

line_re = [re.compile(regex) for regex in (
    r'(Six|Nine) (at|in) the beginning means:',
    r'(Six|Nine)  ?in t?h?e second place means:',
    r'(Six|Nine)  ?in the third place means[:.]',
    r'(Six|Nine) in the fourth place means:',
    r'(Six|Nine) in the fifth place means:',
    r'(Six|Nine) at the top means:',
)]

def capture_line(n, verse):
    hit = line_re[n-1].search(verse)
    if hit is not None:
        return hit.group(1)

hexagram_start = re.compile(r'<a name="\d{1,2}"></a>')
title_parts = re.compile(r'(\d{1,2})\. ([^\n]+)')
chapters = hexagram_start.split(html.unescape(text))
for i, chapter in enumerate(chapters[1:], 1):
    verses = chapter.split('\n')
    hit = title_parts.search(verses[1])
    n, title = hit.groups(verses[1])
    title_wg, title_en = (s.strip() for s in title.split('/'))
    assert i == int(n)
    print(f'{n:2}\t{title_wg:10} {title_en}')
    line_matches = 0
    iter_verses = iter(verses[2:])
    while True:
        try:
            verse = next(iter_verses)
        except StopIteration:
            break
        for n in range(1, 7):
            hit = capture_line(n, verse)
            if hit is not None:
                line = 6 if hit == 'Six' else 9
                print('\t', n, line)
                line_matches += 1
                verse = next(iter_verses)
                while verse.startswith('\t'):
                    print('\t\t', verse)
                    verse = next(iter_verses)
                
    if line_matches != 6:
        raise ValueError('!!! missing line text')

In [ ]:
2